In [ ]:
# Load VGG16_BN model
import os
import torch
from model import VGG16_BN


model = VGG16_BN()
chk_path = os.path.join("out","best.tar")

if os.path.isfile(chk_path):
    print("=> loading checkpoint '{}'".format(chk_path))
    checkpoint = torch.load(chk_path)
    best_prec1 = checkpoint['best_prec1']
    model.load_state_dict(checkpoint['state_dict'])
    print("=> loaded model with accuracy {}".format(best_prec1))
else:
    print("=> no checkpoint found at '{}'".format(chk_path))

=> loading checkpoint 'out/best.tar'
=> loaded model with accuracy 87.82


In [ ]:
# Mask Determination
ratio = 0.07 # of total image
size = 8   # size of trigger

def init_mask():
    mask = torch.zeros(32,32)
    patch = torch.ones(size, size)
    start_x = 22
    start_y = 22
    mask[start_x:start_x+size, start_y:start_y+size] = patch
    return mask

# Layer Determination
def select_layer(model, layer=1):
    for name, param in model.named_parameters():
        if name == "fc{}.weight".format(layer):
            return param.sum(dim=1)

In [ ]:
# Neuron Determination
from torch.utils.data import Dataset
import pickle as pkl
from tqdm import tqdm

layer = 1
target_class = 3
balance_factor = 0.65

class EvalDataset(Dataset):
    def __init__(self, target=1):
        self.inputs = []
        self.target = target
        with open("data/class_{}.tar".format(self.target), 'rb') as f:
            self.inputs = pkl.load(f)
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        return self.inputs[idx]

loader = torch.utils.data.DataLoader(EvalDataset(target_class),batch_size=1, shuffle=True,num_workers=4, pin_memory=True)

weight = select_layer(model, layer=layer)
activated_neurons = None
model.eval()
for i, input in enumerate(tqdm(loader)):
    output, activation = model(input, layer)
    if activated_neurons is None:
        activated_neurons = activation
    else:
        activated_neurons += activation
tmp = activated_neurons*balance_factor + weight*(1-balance_factor)
_, selected_neuron = torch.topk(tmp, 1)

100%|██████████| 5000/5000 [06:08<00:00, 13.58it/s]

torch.Size([4096])
torch.Size([1, 4096])


In [18]:
# Trigger formation
print(max(activated_neurons))

tensor(3889.6387, grad_fn=<SelectBackward>)


In [ ]:
l = layer_selection(model)
Trigger = mask_initialization(mask)
selected_neuron = neuron_selection(model, layer)
selected_activation = neuron_activation(model, layer, neuron)
t_n_i = max_value(model, layer)

cost = dist(selected_activation - t_n_i)
di_f_i = max()